# Insertion Sort vs Bucket Sort: A few test cases to show where each one outperforms the other
This is a short Jupyter Notebook aimed at showing where these two sorting algorithms can excel with respect to one-another. Insertion Sort has quadratic time complexity, while Bucket Sort is linear while a certain condition is true: the elements in the array it is sorting need to be evenly distributed. They operate on the same data types (integers, floats, decimals etc.), and Bucket Sort can be used in every case Insertion Sort is usable, unlike Radix Sort for example. Bucket Sort has one distinct advantage: it can be parallelized, whereas Insertion Sort cannot.

Below are definitions in Python for the sorting algorithms and test cases to give an idea of performance relative to these case conditions.

In [147]:
import itertools
import math
import copy

class SortingUtilities:

    # standard Insertion Sort implementation
    # deep copy is used to prevent caching of original array which would result in faster but false performance for Insertion Sort
    @staticmethod
    def insertion_sort(array):
        current_index = 1
        new_array = copy.deepcopy(array)
        while current_index < len(array):
            other_index = current_index
            while other_index > 0 and new_array[other_index - 1] > new_array[other_index]:
                # swap
                tmp = new_array[other_index]
                new_array[other_index] = new_array[other_index - 1]
                new_array[other_index - 1] = tmp

                other_index = other_index - 1
            current_index = current_index + 1
        
        return new_array
    
    # one possible implementation of Bucket Sort
    # can be made more efficient with respect to space usage and complexity in another lower-level language such as C, C++ or Java
    # an implicit precondition of this implementation is that the number of elements shouldn't be smaller or equal to the greatest number 
    @staticmethod
    def bucket_sort(array, bucket_count):
        buckets = []
        for k in range(0, bucket_count):
            buckets.append([])

        array_length = len(array)
        for i in range(0, array_length):
            position = math.floor(bucket_count * array[i] / array_length)
            buckets[position].append(array[i])
        
        new_array = []
        for j in range(0, bucket_count):
            sorted_bucket = SortingUtilities.insertion_sort(buckets[j])
            new_array = new_array + sorted_bucket
        
        return new_array


In [148]:
# quick test to check if changes in code work
array1 = [5, 1, 4, 2, 7, 1, 6, 8]
array2 = [4, 7, 1, 8, 9, 5, 9, 1, 5, 3, 3]
sorted1 = SortingUtilities.insertion_sort(array1)
sorted2 = SortingUtilities.bucket_sort(array2, 5)

assert(array1 != sorted1)
assert(array2 != sorted2)
assert(sorted1 == [1, 1, 2, 4, 5, 6, 7, 8])
assert(sorted2 == [1, 1, 3, 3, 4, 5, 5, 7, 8, 9, 9])

## Quick Estimation of Time Complexities for each Sorting Algorithm
### Insertion Sort
The outer `while` loop always does `n - 1` runs.
```python
    while current_index < len(array):
        ...
        current_index = current_index + 1
```

The inner `while` loop does on average less than and at most `other_index`, which is by definition between zero and `current_index < n`.
```python
    while other_index > 0 and array[other_index - 1] > array[other_index]:
        ...
        other_index = other_index - 1
```
With these in mind we can estimate the Time Complexity of Insertion Sort `Big O of (n-1) * current_index ~ n^2`, so `O(n^2)`.

Here is a table of complexity by case from Wikipedia (conforming to my implementation):
| Case            | Complexity                         |
|-----------------|------------------------------------|
| Worst-case Time | `O(n^2)` comparisons and swaps     |
| Best-case Time  | `O(n^2)` comparisons, `O(1)` swaps |
| Average Time    | `O(n^2)` comparisons and swaps     |
| Spatial         | `O(n)` (another copy list)         |

### Bucket Sort
There are four `for`` loops. Let's inspect each one and then have a look at the Spatial Complexity.

The first `for` loop does `k` runs, `k` being the number of buckets and a parameter passed during execution by the user.
```python
    for k in range(0, bucket_count):
        ...
```

The second loop evidently does `n` runs.

The third does `k` runs, but inside itself it calls Insertion Sort: `k * (n/k)^2 = n^2/k` runs.
```python
    for j in range(0, bucket_count):
        SortingUtilities.insertion_sort(buckets[j])
        ...
```
The last loop only populates the old array with the sorted values and is optional if a faster implementation is desired. It does `n` runs.

So we can expect the Time Complexity to be `O(k + n + n^2/k)`. `k` can be any integerb between one and `n`. When `k = 1` it is blatanly an Insertion Sort. When `k ~ n`, we have `O(n)`. So using a large number of buckets can be advantageous in terms of time.

Lastly, Spatial Complexity is `O(n + k)`: there is a list of `k` buckets and the buckets themselves contain `n` elements in total.

## Test Cases
We want to show when Insertion Sort excels for arrays of increasing sizes and when Bucket Sort is better than Insertion Sort. To generate evenly distributed populated lists, I defined a static method inside `DataGenerationUtilities`.

Insertion Sort is simple and has low spatial complexity. It is generally used for simple cases where array sizes are not large. A test case for this would be __sorting lists with few elements__.

Insertion Sort's quadratic complexity however means that for a sufficiently big array of numbers, Bucket Sort will be faster than Insertion Sort. A test case for this would obviously be __sorting of very large lists__.

We also want to test whether Bucket Sort is better than Insertion Sort for large arrays where the numbers are not evenly distributed. To generate uneven distributions of elements for the input, I concatenated two lists: one evenly distributed with the full range and another also evenly distributed but with a much more limited range. This effectively gives a list with the full range but with uneven distribution in the limited range of the second list.

Lastly, we want to test what number of buckets is best relative to array size. For this we will test different numbers of buckets.

## Note on Results
I ran the cells locally to get some results and noted these under each test case. The time it takes to execute the algorithm is calculated with [this](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit), called `timeit`.

I specified that it time the algorithms by taking 10 samples (`-r 10`, so 10 repeats) and for each sample or repeated timing loop, it should do 10 loops (`-n 10`). The reason for this choice is because `timeit` generates statistical data for the time samples, but there is a bit of overhead for the initiation of each sampling take. So in order to "cancel out" the overhead, we do a few internal loops inside each sampling take. Obviously, higher values for both `r` and `n` give statistically more sound results. 

In [149]:
import random

class DataGenerationUtilities:

    @staticmethod
    def generate_evenly_distributed_integer_list(size, max_value):
        return [random.randint(0, max_value) for i in range(size)]

    @staticmethod
    def generate_repeated_integer_list(size, value):
        return [value for i in range(size)]

### Test Case 1: Evenly Distributed Array of 10 Elements Sorted

In [328]:
tc1_array1 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10, 9)
tc1_array2 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10, 9)
tc1_array3 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10, 9)

In [297]:
%%timeit -r 10 -n 10
SortingUtilities.insertion_sort(tc1_array1)

5.6 µs ± 459 ns per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [329]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc1_array2, 5)

10.5 µs ± 1.71 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [324]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc1_array3, 3)

6.98 µs ± 820 ns per loop (mean ± std. dev. of 10 runs, 10 loops each)


My results locally:
* Insertion Sort: `6.26 µs ± 658 ns per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with 5 buckets (~20 elements per bucket): `9.05 µs ± 869 ns per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with 3 buckets: `6.98 µs ± 820 ns per loop (mean ± std. dev. of 10 runs, 10 loops each)`

### Test Case 2: Evenly Distributed Array of 100 Elements Sorted

In [300]:
tc2_array1 = DataGenerationUtilities.generate_evenly_distributed_integer_list(100, 99)
tc2_array2 = DataGenerationUtilities.generate_evenly_distributed_integer_list(100, 99)
tc2_array3 = DataGenerationUtilities.generate_evenly_distributed_integer_list(100, 99)

In [301]:
%%timeit -r 10 -n 10
SortingUtilities.insertion_sort(tc2_array1)

215 µs ± 14.1 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [302]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc2_array2, 5)

82.1 µs ± 1.59 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [303]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc2_array3, int(math.sqrt(len(tc2_array3))))

64.8 µs ± 3.27 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


My results:
* Insertion Sort: `229 µs ± 12.3 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with 50 buckets (~20 elements per bucket): `88.4 µs ± 2.19 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with square root of `n` buckets (~10 elements per bucket): `67.8 µs ± 2.21 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)`

### Test Case 3: Evenly Distributed Array of 10000 Elements Sorted

In [315]:
tc3_array1 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10000, 9999)
tc3_array2 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10000, 9999)
tc3_array3 = DataGenerationUtilities.generate_evenly_distributed_integer_list(10000, 9999)

In [316]:
%%timeit -r 10 -n 10
SortingUtilities.insertion_sort(tc3_array1)

2.52 s ± 11 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [312]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc3_array2, 10)

31.6 µs ± 2.03 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [311]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc3_array3, int(math.sqrt(len(tc3_array3))))

33.7 µs ± 6.37 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


My results:
* Insertion Sort: `2.24 s ± 6.46 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with 5000 buckets (~20 elements per bucket): `12.6 ms ± 194 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with square root of `n` buckets (~100 elements per bucket): `24.9 ms ± 161 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)`

### Test Case 4: Unevenly Distributed Array of 10000 Elements Sorted

In [317]:
tc4_array1 = DataGenerationUtilities.generate_evenly_distributed_integer_list(9000, 9) + DataGenerationUtilities.generate_evenly_distributed_integer_list(1000, random.randint(10, 9999))
tc4_array2 = DataGenerationUtilities.generate_evenly_distributed_integer_list(9000, 9) + DataGenerationUtilities.generate_evenly_distributed_integer_list(1000, random.randint(10, 9999))
tc4_array3 = DataGenerationUtilities.generate_evenly_distributed_integer_list(9000, 9) + DataGenerationUtilities.generate_evenly_distributed_integer_list(1000, random.randint(10, 9999))

In [ ]:
%%timeit -r 10 -n 10
SortingUtilities.insertion_sort(tc4_array1)

In [319]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc4_array2, 500)

1.63 s ± 5.29 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [277]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc4_array3, int(math.sqrt(len(tc4_array3))))

1.67 s ± 58.8 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


My results:
* Insertion Sort: `1.84 s ± 28.5 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with 500 buckets (~20 elements per bucket): `1.74 s ± 12 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with square root of `n` buckets (~100 elements per bucket): `1.67 s ± 58.8 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`

### Test Case 5: The Importance of Parameter Choice on Bucket Sort

In [320]:
tc5_array1 = DataGenerationUtilities.generate_evenly_distributed_integer_list(60000, 9999)
tc5_array2 = DataGenerationUtilities.generate_evenly_distributed_integer_list(60000, 9999)

In [322]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc5_array1, 3000)

516 ms ± 2.08 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [323]:
%%timeit -r 10 -n 10
SortingUtilities.bucket_sort(tc5_array2, int(math.sqrt(len(tc5_array2))))

2.02 s ± 15.4 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


My results:
* Bucket Sort with 3000 buckets (~20 elements per bucket): `516 ms ± 2.08 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`
* Bucket Sort with square root of `n` buckets (~250 elements per bucket): `2.02 s ± 15.4 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)`

## Interpretation & Final Remarks
For small lists (10 to 20 elements) Insertion Sort is faster than Bucket Sort. At another order of magnitude of elements higher (100), Insertion Sort is somewhat slower with respect to Bucket Sort. Under other circumstances, this could have been attributed to the memory allocation overhead (our implementation performs a deep copy of the original list so that successive runs for the same list aren't faster); however, our Bucket Sort implementation uses this same deep copy technique becuase it uses the same Insertion Sort algorithm inside. This means that Bucket Sort is faster in these conditions regardless of the extra overhead, because this overhead is common to both implementations.  We also see that Bucket Sort is around two order of magnitude faster for lists of 10 thousand elements.

The above results were under assumption of evenly distributed lists (generated via `random.randint()`, random enough for these experiments). When ran against lists of 10 thousand elements of a very uneven distribution (0.9 for [0,9] and 0.1 [10,9999]) Insertion Sort was almost as fast as Bucket Sort. This is a known limition of Bucket Sort, as one of its preconditions is that the input should be uniformly distributed.

Lastly, you should have noticed that Bucket Sort was ran with two different parameters for the number of buckets. The first parameter was chosen so that buckets would be of size 20. This is because Insertion Sort is known to perform better for small lists, and also as a side effect gives us a large number of buckets which in reference to our analysis of complexity should result in a linear time complexity. The second parameter was chosen to be approximately the square root of the input size. This would theoretically give a time complexity of `n * sqrt(n)`, which is still better than the quadratic time complexity of Insertion Sort and serves for a nice comparison against the first parameter choice as well. Unsurprisingly, our hypothesis holds true: Bucket Sort is faster when ran with the first parameter for large input sizes.

## Appendix: Code Documentation
`SortingUtilities.insertion_sort(array)`: standard Insertion Sort implementation. `array` should be a Python list.

`SortingUtilities.bucket_sort(array, bucket_count)`: Bucket Sort implementation. Creates `bucket_count` buckets. Elements are inserted into said buckets via `position = math.floor(bucket_count * array[i] / array_length)`. A side effect of this is that the number of elements shouldn't be smaller or equal to the greatest number. Indeed, the user should get an `IndexError` if this precondition on the input is not satisfied. `array` should be a Python list, and `bucket_count` an integer.

`DataGenerationUtilities.generate_evenly_distributed_integer_list(size, max_value)`: generates a list of integers between 0 and `max_value` inclusive, of length `size`. `max_value` can any positive number, while `size` must be an integer. The distribution of the list is that of `random.randint()`, which is deterministic (pseudo-random) but statistically uniform[^](https://docs.python.org/3/library/random.html).
